In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

2021-10-27 13:54:30.623774: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import gc
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, LSTM, GRU
from tensorflow.keras.layers import Bidirectional, Multiply

np.random.seed(42)
tf.random.set_seed(42)

In [3]:

submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')


In [4]:
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K
class WarmupExponentialDecay(Callback):
    def __init__(self,lr_base=0.0002,lr_min=0.0,decay=0,warmup_epochs=0):
        self.num_passed_batchs = 0   #一个计数器
        self.warmup_epochs=warmup_epochs  
        self.lr=lr_base #learning_rate_base
        self.lr_min=lr_min #最小的起始学习率,此代码尚未实现
        self.decay=decay  #指数衰减率
        self.steps_per_epoch=0 #也是一个计数器
        
    def on_batch_begin(self, batch, logs=None):
        # params是模型自动传递给Callback的一些参数
        if self.steps_per_epoch==0:
            #防止跑验证集的时候呗更改了
            if self.params['steps'] == None:
                self.steps_per_epoch = np.ceil(1. * self.params['samples'] / self.params['batch_size'])
            else:
                self.steps_per_epoch = self.params['steps']
        if self.num_passed_batchs < self.steps_per_epoch * self.warmup_epochs:
            K.set_value(self.model.optimizer.lr,
                        self.lr*(self.num_passed_batchs + 1) / self.steps_per_epoch / self.warmup_epochs)
        else:
            K.set_value(self.model.optimizer.lr,
                        self.lr*((1-self.decay)**(self.num_passed_batchs-self.steps_per_epoch*self.warmup_epochs)))
        self.num_passed_batchs += 1
        
    def on_epoch_begin(self,epoch,logs=None):
        #用来输出学习率的,可以删除
        print("learning_rate:",K.get_value(self.model.optimizer.lr))

In [5]:
import gc
gc.collect()

43

In [6]:
test = np.load('/kaggle/input/vpp-data-processed/test_data.npy')

In [7]:
shp = test.shape[-2:]
shp

(80, 50)

In [8]:
home = 'vpp-data-processed'
FOLD = 0
EPOCH = 200
BATCH_SIZE = 512

# detect and init the TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
for i in range(1):
#with tpu_strategy.scope():
    #kf = KFold(n_splits=NUM_SPLITS, shuffle=True, random_state=2021)
    test_preds = []
    for FOLD in range(5):
        
        print('----'*15, '>', f'Fold {FOLD}', '<', '----'*15)
        X_train, X_valid = np.load(f'/kaggle/input/{home}/X_train_{FOLD}.npy'),np.load(f'/kaggle/input/{home}/X_valid_{FOLD}.npy')
        y_train, y_valid = np.load(f'/kaggle/input/{home}/y_train_{FOLD}.npy'),np.load(f'/kaggle/input/{home}/y_valid_{FOLD}.npy')
        
        model = keras.models.Sequential([
            keras.layers.Input(shape=X_train.shape[-2:]),
            keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
            keras.layers.Dense(64, activation='selu'),
            keras.layers.Dense(1),
        ])
        model.compile(optimizer="adam", loss="mae")

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
        
        es = EarlyStopping(monitor="val_loss", patience=60, verbose=1, mode="min", restore_best_weights=True)
    
        checkpoint_filepath = f"best_model_folds{FOLD}.h5"
        
        sv = keras.callbacks.ModelCheckpoint(
            checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
            save_weights_only=False, mode='auto', save_freq='epoch',
            options=None
        )

        model.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
                  epochs=EPOCH, 
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, es, sv])
        
        del X_train,X_valid,y_train,y_valid
        gc.collect()
        
        test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

------------------------------------------------------------ > Fold 0 < ------------------------------------------------------------


2021-10-27 13:55:01.593917: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-27 13:55:01.596923: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-27 13:55:01.639451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-27 13:55:01.640119: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-27 13:55:01.640186: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-27 13:55:01.666933: I tensorflow/stream_executor/platform/def

Epoch 1/200


2021-10-27 13:55:17.631376: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-10-27 13:55:18.468885: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-10-27 13:55:18.534477: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


118/118 [==============================] - ETA: 0s - loss: 5.0784

...

------------------------------------------------------------ > Fold 1 < ------------------------------------------------------------


2021-10-27 14:40:34.163536: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 965760000 exceeds 10% of free system memory.


Epoch 1/200
118/118 [==============================] - ETA: 0s - loss: 4.8516

2021-10-27 14:40:58.482629: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 241440000 exceeds 10% of free system memory.


118/118 [==============================] - 27s 134ms/step - loss: 4.8352 - val_loss: 1.1624

Epoch 00001: val_loss improved from inf to 1.16239, saving model to best_model_folds1.h5
Epoch 2/200
118/118 [==============================] - 13s 113ms/step - loss: 1.0592 - val_loss: 0.9149

Epoch 00002: val_loss improved from 1.16239 to 0.91491, saving model to best_model_folds1.h5
Epoch 3/200
118/118 [==============================] - 13s 114ms/step - loss: 0.8815 - val_loss: 0.7532

Epoch 00003: val_loss improved from 0.91491 to 0.75319, saving model to best_model_folds1.h5
Epoch 4/200
118/118 [==============================] - 12s 106ms/step - loss: 0.7137 - val_loss: 0.6555

Epoch 00004: val_loss improved from 0.75319 to 0.65550, saving model to best_model_folds1.h5
Epoch 5/200
118/118 [==============================] - 13s 113ms/step - loss: 0.6385 - val_loss: 0.6175

Epoch 00005: val_loss improved from 0.65550 to 0.61748, saving model to best_model_folds1.h5
Epoch 6/200
118/118 [=====

In [9]:
PRESSURE_MAX,PRESSURE_MIN,PRESSURE_STEP = 64.82099173863948, -1.8957442945646408, 0.07030214545120961

In [10]:
submission["pressure"] = np.median(np.vstack(test_preds),axis=0)

In [11]:
submission["pressure"] =\
    np.round( (submission.pressure - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN
submission.pressure = np.clip(submission.pressure, PRESSURE_MIN, PRESSURE_MAX)

In [12]:
submission.to_csv('submission_median_round.csv', index=False)